In [1]:
function sel(n::Int64)
    return list -> list[n]
end

S1 = sel(1)
S2 = sel(2)
S3 = sel(3)

(::#1) (generic function with 1 method)

In [2]:
function larBernsteinBasis(u::Function, n::Int64)
    function map_fn(x::Array{Float64,1})
        bernstein(i, n, t::Float64) = binomial(n,i)*(1-t)^(n-i)*t^i
        return [bernstein(i, n, u(x)) for i in 0:n]
    end
    return map_fn
end

larBernsteinBasis (generic function with 1 method)

In [3]:
#--UNIT TEST--#

bernstein = larBernsteinBasis(S1, 3)
point = [0.7,2,7.3]
result = bernstein(point)

4-element Array{Float64,1}:
 0.027
 0.189
 0.441
 0.343

In [10]:
function larBezier{T<:Real}(u::Function, controldata::Array{Array{T,1},1})
    n = length(controldata) - 1
    dim = length(controldata[1])
    function map_fn(point::Array{Float64, 1})
        t = u(point)
        out = zeros(Float64, dim)
        for i in 0:n
            weight = binomial(n,i)*(1-t)^(n-i)*t^i
            for k in 1:dim
                out[k] += weight*controldata[i+1][k]
            end
        end
        return out
    end
    return map_fn
end

larBezier (generic function with 1 method)

In [11]:
#--UNIT TEST--#

controldata = [[0,1],[0,0],[1,1],[1,0]]
point = [0.5, 0.7, 1.2]
u = S2
bezier = larBezier(u, controldata)
result = bezier(point)

2-element Array{Float64,1}:
 0.784
 0.468

In [6]:
function larBezierCurve(controlpoints)
   return larBezier(S1, controlpoints)
end

larBezierCurve (generic function with 1 method)

In [7]:
#--UNIT TEST--#

controls = [[0,1],[0,0],[1,1],[1,0]]
point = [0.2, 0.7, 1.2]
bezier = larBezierCurve(controls)
result = bezier(point)

2-element Array{Float64,1}:
 0.104
 0.608

In [8]:
function larTensorProdSurface{F<:Function, T<:Real}(args::Array{F, 1}, 
                            controlpoints::Array{Array{Array{T, 1}, 1}, 1})
    ubasis , vbasis = args
    function map_fn(point::Array{Float64, 1})
        u, v = point
        U = ubasis([u])
        V = vbasis([v])
        target_dim = length(controlpoints[1][1])
        ret = zeros(Float64, target_dim)
        dim_u = length(U)
        dim_v = length(V)
        if (length(controlpoints[1])!= dim_u || length(controlpoints[2])!= dim_v)
            error("Invalid set of control points for those bases!") 
        end
        for i in 1:dim_u
            for j in 1:dim_v
                #for M in 1:target_dim
                    for M in 1:target_dim 
                        ret[M] += U[i]*V[j]*controlpoints[i][j][M]
                    end
                #end
            end
        end
        return ret
    end
    return map_fn
end

larTensorProdSurface (generic function with 1 method)

In [9]:
#--UNIT TEST--#

controlpoints = [[[0,0,0],[2,-4,2]],[[0,3,1],[4,0,0]]]
basis = larBernsteinBasis(S1, 1)
point = [0.5, 2]
tensorprodsurf = larTensorProdSurface([basis, basis], controlpoints)
result = tensorprodsurf(point)

3-element Array{Float64,1}:
  6.0
 -5.5
  1.5

In [16]:
function larBilinearSurface{T<:Real}(controlpoints::Array{Array{Array{T, 1}, 1}, 1})
   basis = larBernsteinBasis(S1, 1)
   return larTensorProdSurface([basis, basis], controlpoints)
end

larBilinearSurface (generic function with 1 method)

In [17]:
#--UNIT TEST--#

controlpoints = [[[0,0,0],[2,-4,2]],[[0,3,1],[4,0,0]]]
point = [0.5,0.7]
bilinear = larBilinearSurface(controlpoints)
result = bilinear(point)

3-element Array{Float64,1}:
  2.1 
 -0.95
  0.85

In [18]:
function larBiquadraticSurface{T<:Real}(controlpoints::Array{Array{Array{T, 1}, 1}, 1})
   basis1 = larBernsteinBasis(S1, 2)
   basis2 = larBernsteinBasis(S1, 2)
   return larTensorProdSurface([basis1,basis2], controlpoints)
end

larBiquadraticSurface (generic function with 1 method)

In [20]:
#--UNIT TEST--#

controlpoints = [[[0,0,0],[2,0,1],[3,1,1]],
               [[1,3,-1],[2,2,0],[3,2,0]],
               [[-2,4,0],[2,5,1],[1,3,2]]]
biquadratic = larBiquadraticSurface(controlpoints)
point = [0.7, 0.2]
result = biquadratic(point)

3-element Array{Float64,1}:
  0.3624
  3.2096
 -0.0404

In [21]:
function larBicubicSurface{T<:Real}(controlpoints::Array{Array{Array{T, 1}, 1}, 1})
   basis1 = larBernsteinBasis(S1, 3)
   basis2 = larBernsteinBasis(S1, 3)
   return larTensorProdSurface([basis1, basis2], controlpoints)
end

larBicubicSurface (generic function with 1 method)

In [22]:
#--UNIT TEST--#

controlpoints = [[[0,0,0], [0,3,4], [0,6,3], [0,10,0]],
                 [[3,0,2], [2,2.5,5], [3,6,5], [4,8,2]],
                 [[6,0,2], [8,3,5], [7,6,4.5], [6,10,2.5]],
                 [[10,0,0], [11,3,4], [11,6,3], [10,9,0]]]
bicubic = larBicubicSurface(controlpoints)
point = [0.7,0.2]
result = bicubic(point)

3-element Array{Float64,1}:
 7.1176 
 1.76594
 2.82268

In [25]:
function larCoonsPatch{T<:Function}(args::Array{T, 1})
    su0_fn , su1_fn , s0v_fn , s1v_fn = args
    function map_fn(point)
        u, v = point
        su0 = su0_fn(point)
        su1 = su1_fn(point)
        s0v = s0v_fn(point)
        s1v = s1v_fn(point)
        ret = zeros(Float64, length(su0))  
        for k in 1:length(ret)
            ret[k] = ((1-u)*s0v[k] + u*s1v[k]+(1-v)*su0[k] + v*su1[k] + 
                     (1-u)*(1-v)*s0v[k] + (1-u)*v*s0v[k] + u*(1-v)*s1v[k] + u*v*s1v[k])
        end
        return ret
    end
    return map_fn
end

larCoonsPatch (generic function with 1 method)

In [26]:
#--UNIT TEST--#

Su0 = larBezier(S1, [[0,0,0],[10,0,0]])
Su1 = larBezier(S1, [[0,10,0],[2.5,10,3],[5,10,-3],[7.5,10,3],[10,10,0]])
Sv0 = larBezier(S2, [[0,0,0],[0,0,3],[0,10,3],[0,10,0]])
Sv1 = larBezier(S2, [[10,0,0],[10,5,3],[10,10,0]])
coons = larCoonsPatch([Su0,Su1,Sv0,Sv1])
point = [0.2, 0.7]
coons(point)

3-element Array{Float64,1}:
  6.0    
 22.344  
  4.11936

In [23]:
function bsplineBasis{T<:Real}(degree::Int64, knots::Array{T,1}, ncontrols::Int64)
    n = ncontrols - 1
    m = length(knots) - 1
    k = degree + 1
    tmin, tmax = knots[k-1], knots[n+1]       
    if length(knots)!=(n+k+1) 
        error("Invalid point/knots/degree for bspline!") 
    end
    # de Boor coefficients
    function deBoor(i::Int64, k::Int64, t::Float64)           
        # deBoor coefficient Ni1(t)
        if k == 1
            if(t >= knots[i] && t<knots[i+1]) || (t == tmax && t>=knots[i] && t<=knots[i+1])
                # t in [T_i, T_(i+1)) vel t=T_(n+1) in [T_i, T_(i+1)]
                return 1
            else
                return 0
            end
        end
        # deBoor coefficient Nik(t)
        ret = 0    
        num1, div1 = t-knots[i], knots[i+k-1]-knots[i]
        if div1 != 0 
            ret += (num1/div1) * deBoor(i,k-1,t) 
        end
        num2, div2 = knots[i+k]-t, knots[i+k]-knots[i+1]
        if div2 != 0
            ret += (num2/div2) * deBoor(i+1,k-1,t)
        end
        return ret
    end
    # map function
    function map_fn{T<:Real}(point::Array{T, 1})
        t = point[1]
        return [deBoor(i,k,t) for i in 1:n+1]
    end
    return map_fn
end

bsplineBasis (generic function with 1 method)

In [24]:
#--UNIT TEST--##

knots = [0,0,0,0,1,2,3,4,4,4,4]
ncontrols = 8
degree = 2
basis = bsplineBasis(degree, knots, ncontrols)
point = [0.7,0.2,3.0]
basis(point)

8-element Array{Real,1}:
 0    
 0.09 
 0.665
 0.245
 0.0  
 0.0  
 0.0  
 0    

In [27]:
function t_bspline{T<:Real, F<:Function}(u::Function, degree::Int64, knots::Array{T,1}, 
                                    points_fn::Array{F, 1})
   
    n = length(points_fn)-1
    m = length(knots)-1
    k = degree + 1
    tmin,tmax = knots[k-1], knots[n+1]
   
    # see http://www.na.iac.cnr.it/~bdv/cagd/spline/B-spline/bspline-curve.html
    if length(knots)!=(n+k+1) 
        error("Invalid point/knots/degree for bspline!") 
    end
    # de Boor coefficients
    function deBoor(i::Int64, k::Int64, t::Float64)
        # Ni1(t)
        if k == 1 
            if(t>=knots[i] && t<knots[i+1]) || (t==tmax && t>=knots[i] && t<=knots[i+1])
                # i use strict inclusion for the max value
                return 1
            else
                return 0
            end
        end
        # Nik(t)
        ret = 0
        
        num1, div1 = t-knots[i], knots[i+k-1]-knots[i]  
        if div1 != 0 
            ret+=(num1/div1) * deBoor(i,k-1,t)
            # elif num1!=0: ret+=deBoor(i,k-1,t)
        end
        
        num2, div2 = knots[i+k]-t, knots[i+k]-knots[i+1]
        if div2!=0  
            ret+=(num2/div2) * deBoor(i+1,k-1,t)
            # elif num2!=0: ret+=deBoor(i,k-1,t)
        end
           
        return ret
    end
    
    # map function
    function map_fn(point::Array{Float64, 1})
        t = u(point)
        # if control points are functions
        #points = point -> [f(point) for f in points_fn]    #POSSIBILE ERRORE QUI
        target_dim = length(points_fn[1](point))    #errore qui
        ret = zeros(Float64, target_dim)
        for i in 1:n+1
            coeff = deBoor(i,k,t) 
            for m in 1:target_dim
                ret[m] += points_fn[i](point)[m] * coeff
            end
        end
        return ret
    end
    return map_fn
end

t_bspline (generic function with 1 method)

In [29]:
#--UNIT TEST--#

b1 = larBezier(S1, [[0,1,0],[0,1,5]])
b2 = larBezier(S1, [[0,0,0],[0,0,5]])
b3 = larBezier(S1, [[1,0,0],[2,-1,2.5],[1,0,5]])
b4 = larBezier(S1, [[1,1,0],[1,1,5]])
b5 = larBezier(S1, [[0,1,0],[0,1,5]])
controls = [b1, b2, b3, b4, b5]
knots = [0,1,2,3,4,5,6,7]           # periodic B-spline
knots = [0,0,0,1,2,3,3,3]           # non-periodic B-spline
tbspline = t_bspline(S2, 2, knots, controls) 
point = [0.7, 0.2]
result = tbspline(point)

3-element Array{Float64,1}:
 0.0284
 0.6316
 3.5   

In [30]:
function t_nurbs{T<:Real, F<:Function}(u::Function, degree::Int64, knots::Array{T,1}, 
                                       points::Array{F, 1})
    bspline = t_bspline(u, degree, knots, points)
    function map_fn(point)          #eliminare
        ret = bspline(point)
        last = ret[end]  #last element in ret
        if last != 0
            ret = [value/last for value in ret]
        end
        ret = ret[1:end-1] #it slices off the last element (that is now 1)
        return ret
    end
    return map_fn
end

t_nurbs (generic function with 1 method)

In [31]:
#--UNIT TEST--#

knots = [0,0,0,1,1,2,2,3,3,4,4,4]
p = sqrt(2)/2.0
controls = [[-1,0,1], [-p,p,p], [0,1,1], [p,p,p],[1,0,1], [p,-p,p], 
         [0,-1,1], [-p,-p,p], [-1,0,1]]
c1 = larBezier(S1, [[-1,0,0,1],[-1,0,1,1]])
c2 = larBezier(S1, [[-p,p,0,p],[-p,p,p,p]])
c3 = larBezier(S1, [[0,1,0,1],[0,1,1,1]])
c4 = larBezier(S1, [[p,p,0,p],[p,p,p,p]])
c5 = larBezier(S1, [[1,0,0,1],[1,0,1,1]])
c6 = larBezier(S1, [[p,-p,0,p],[p,-p,p,p]])
c7 = larBezier(S1, [[0,-1,0,1],[0,-1,1,1]])
c8 = larBezier(S1, [[-p,-p,0,p],[-p,-p,p,p]])
c9 = larBezier(S1, [[-1,0,0,1],[-1,0,1,1]])
controls = [c1,c2,c3,c4,c5,c6,c7,c8,c9]
         
tnurbs = t_nurbs(S2,2,knots,controls)
point = [0.7, 0.2]
result = tnurbs(point)

3-element Array{Float64,1}:
 -0.955863
  0.293812
  0.7     